# Homework 10

- Python Computing for Data Science (2016)

- Due Monday November 28 (5pm)

1) Write a module called ‘CalCalc’, with a method called ‘calculate’ that evaluates any string passed to it, and can be used from either the command line (using argparse with reasonable flags) or imported within Python. Feel free to use something like eval(), but be aware of some of the nasty things it can do, and make sure it doesn’t have too much power:  http://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html

EXAMPLE:
```bash
$ python CalCalc.py -s '34*28'
$ 952
```
 AND, from within Python
 
```python
>>> from CalCalc import calculate
>>> calculate('34*20')
>>> 952
```
1a) To make this more awesome, have your function interact with the Wolfram|Alpha API to ask it what it thinks of the difficult questions.  To make this work, experiment with urllib2 and a URL like this:
'http://api.wolframalpha.com/v2/query?input=XXXXX&appid=UAGAWR-3X6Y8W777Q'
where you replace the XXXXX with what you want to know.  NOTE: the ‘&appid=UAGAWR-3X6Y8W777Q’ part is vital; it is a W|A AppID I got for the class.  Feel free to use that one, or you can get your own and read more about the API, here:   http://products.wolframalpha.com/api/
And you can explore how it works here:  http://products.wolframalpha.com/api/explorer.html

EXAMPLE:
```bash
$ python CalCalc.py 'mass of the moon in kg'
$  7.3459e+22
```
AND, from within Python
```python
>>> from CalCalc import calculate
>>> calculate('mass of the moon in kg',  return_float=True) * 10
>>> 7.3459e+23
```

In [96]:
%%writefile CalCalc.py

import argparse
import urllib
import urllib.request
import xml.etree.ElementTree as ET

parser = argparse.ArgumentParser(description='Sample Application')
parser.add_argument('required_arg_1', help='This positional argument is required')

results = parser.parse_args()
incoming = results.required_arg_1

words = incoming
bits = (words.split(' '))
query = ('+'.join(bits))

url = "http://api.wolframalpha.com/v2/query?input="+query+"&appid=APLTT9-9WG78GYE65"

xml_data=urllib.request.urlopen(url).read()
root = ET.fromstring(xml_data)

    output = []
    
    for pt in root.findall('.//plaintext'):
        if pt.text:
            output.append(pt.text)
    if "=" in output[0]: #sometimes result has original query, this gets rid of that
        answer = output[0].split("=")[-1]
        print(answer)
    elif len(output[1].split(" "))>2:
        answer = output[1].split(" ")[0]
        if answer.find("×10^") != -1: #changes scientific notation so python can read it (x10^ -> e)
            parts = answer.split("×10^")
            return(float(parts[0]+"e"+parts[1]))
    if output[1].find("...") != -1: #deals with answers trailing with a '...' (like sqrts, pi)
        shorter = output[1].split("...")
        return(float(shorter[0]))
    else: 
        print("hey")
        return(float((output[1])))

Overwriting CalCalc.py


In [98]:
import urllib
import urllib.request
import xml.etree.ElementTree as ET

def calculate(query_string):
    words = query_string
    bits = (words.split(' '))
    query = ('+'.join(bits))

    url = "http://api.wolframalpha.com/v2/query?input="+query+"&appid=APLTT9-9WG78GYE65"

    xml_data=urllib.request.urlopen(url).read()
    root = ET.fromstring(xml_data)

    output = []
    
    for pt in root.findall('.//plaintext'):
        if pt.text:
            output.append(pt.text)
    if "=" in output[0]: #sometimes result has original query, this gets rid of that
        answer = output[0].split("=")[-1]
        print(answer)
    elif len(output[1].split(" "))>2:
        answer = output[1].split(" ")[0]
        if answer.find("×10^") != -1: #changes scientific notation so python can read it (x10^ -> e)
            parts = answer.split("×10^")
            return(float(parts[0]+"e"+parts[1]))
    if output[1].find("...") != -1: #deals with answers trailing with a '...' (like sqrts, pi)
        shorter = output[1].split("...")
        return(float(shorter[0]))
    else: 
        return(float((output[1])))


In [99]:
def test_1():
    assert abs(4. - calculate(‘2**2’)) < .001
    
def test_2():
    assert type(calculate("mass of the moon in kg")) == str
    
def test_3():
    assert calculate("5x^3") == "15 x^2"

def test_4():
    assert 1 < calculate("sqrt 3") < 2
    
def test_5():
    assert calculate("32*47") == 1504

4.0

2) Start a github project for CalCalc. Include a setup.py, README.txt, LICENSE.txt etc. and turn your module into a proper Python Distribution, so that we can install it and use it.  

Example Folder Hierarchy:
```bash
Your_Homework1_Folder/calcalc
                      |--> CalCalc.py
                      |--> __init__.py
Your_Homework1_Folder/setup.py
Your_Homework1_Folder/README.txt
```
Include at least 5 test functions in CalCalc.py, and test with nosetest, to make sure it behaves the way you think it should.

EXAMPLE CalCalc.py:
```python
# ...
def calculate([...]):
    [...]

def test_1():
    assert abs(4. - calculate(‘2**2’)) < .001
```

When grading, we will create a virtual environment and attempt to install your module by running:

```bash
python setup.py install
```

3) Get your project working with Travis and make sure your tests are run and pass.

3) **(Bonus/Extra Credit)** 

  Get your project working on Azure, AWS or Google Compute Cloud with a Flask front-end. Start a VM on one of these PaaS. A user should be able to submit their calcalc query on a form (hosted on your VM) and get the result back.

You should be able to add an `app.py` (with Flask) into your CalCalc project. Be sure to open up the port on the VM that you are serving on.